This notebook loads data including QQQ and has two modes to run. Generated data with ranks with different parameters so in the end it has multilvel index depending on the parameters  - lookback period X, lookback period Y and so on. then generated strategy based on different stocks (strategy is simple , just buy best x stocks) and final output is simulation and heatmap.

As for the second mode, it includes QQQ, strategy is changed to take consideration of QQQ. QQQ ranks are calcualted as other ranks, and if during the simulation, QQQ ranks is zero than either we don;t do anything until it gets better (positive again) or sell everythig, re buy other once it's get better.


Data is bi-weekly

In [9]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import vectorbt as vbt
from numba import njit
import plotly as plt
from vectorbt.portfolio import nb
import openpyxl
import quantstats as qs
from pandas_datareader import data as pdr
import yfinance as yfin
import glob
from csv import reader
import time
import requests
from bs4 import BeautifulSoup

vbt.settings["caching"]["enabled"] = False
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.options.display.max_seq_items = 2000

In [10]:
#!pip install yfinance==0.2.58

In [11]:
dataset = "QQQ" #should be either "QQQ" or "SNP"
single_params = {"lookback_period":250, "roc_period": 250, "stocks":2}
# Vanilla stays with 1 stock, Correl gets 2 Stock, 250/250  is the best.
multi_params = {"min_lookback_period": 100, "max_lookback_period":300,
                "min_roc_period":100, "max_roc_period": 300,
               "step": 50, "min_stocks":5, "max_stocs": 5}

simulation_type = "single" #should be either "single" or "multi"
wild = False
update = True

In [12]:
data = None
qqq_rank = None

In [13]:
def get_nasdaq_100_symbols():
    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    
    # 1. Add a User-Agent header to mimic a browser
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    
    try:
        # 2. Pass the headers in the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 3. Find the table with the 'Ticker' column
        tables = soup.find_all('table', {'class': 'wikitable'})
        
        target_table = None
        for table in tables:
            headers_text = [th.text.strip() for th in table.find_all('th')]
            if "Ticker" in headers_text:
                target_table = table
                break
        
        if not target_table:
            print("Could not find the components table.")
            return []

        symbols = []
        for row in target_table.find_all('tr')[1:]:
            cols = row.find_all(['td', 'th'])
            if cols:
                ticker = cols[0].text.strip()
                symbols.append(ticker)
        
        return symbols

    except requests.exceptions.RequestException as e:
        print(f"Error fetching page: {e}")
        return []

nasdaq_symbols = get_nasdaq_100_symbols()
    
print(f"Found {len(nasdaq_symbols)} symbols:")
print(nasdaq_symbols)

Found 101 symbols:
['ADBE', 'AMD', 'ABNB', 'ALNY', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'AAPL', 'AMAT', 'APP', 'ARM', 'ASML', 'AZN', 'TEAM', 'ADSK', 'ADP', 'AXON', 'BKR', 'BKNG', 'AVGO', 'CDNS', 'CHTR', 'CTAS', 'CSCO', 'CCEP', 'CTSH', 'CMCSA', 'CEG', 'CPRT', 'CSGP', 'COST', 'CRWD', 'CSX', 'DDOG', 'DXCM', 'FANG', 'DASH', 'EA', 'EXC', 'FAST', 'FER', 'FTNT', 'GEHC', 'GILD', 'HON', 'IDXX', 'INSM', 'INTC', 'INTU', 'ISRG', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LIN', 'MAR', 'MRVL', 'MELI', 'META', 'MCHP', 'MU', 'MSFT', 'MSTR', 'MDLZ', 'MPWR', 'MNST', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'ODFL', 'PCAR', 'PLTR', 'PANW', 'PAYX', 'PYPL', 'PDD', 'PEP', 'QCOM', 'REGN', 'ROP', 'ROST', 'STX', 'SHOP', 'SBUX', 'SNPS', 'TMUS', 'TTWO', 'TSLA', 'TXN', 'TRI', 'VRSK', 'VRTX', 'WBD', 'WDC', 'WDAY', 'XEL', 'ZS']


In [14]:
if update == True:
    # Data update
    #outdated...
    #yfin.pdr_override()
    # open file in read mode
    from1 = "2023-12-02"
    to = "2025-11-04"
    #Delete all
    fileList = glob.glob(r'/home/jj/Stocks/Data/QQQ/*.csv')
    
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)

    if wild == True:
        paf = r'/home/jj/Stocks/Data/WILD.csv'
    else:
        paf = r'/home/jj/Stocks/Data/NASDAQ.csv'
    
    #Download all
    # pass the file object to reader() to get the reader object
    csv_reader = nasdaq_symbols
    # Iterate over each row in the csv using reader object
    downloads = {}
    t = 1
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        string = ''
        path = r'/home/jj/Stocks/Data/QQQ/'
        print(row)
        
        try:
            hist = yfin.download(string.join(row), from1, to)
            #time. sleep(3)
            #hist = web.DataReader(string.join(row), 'yahoo', from1, to)
        except:
            print("Not available with Yahoo: ",row)

        downloads[t] = len(hist)
        if downloads[t] == downloads[1]:
            hist.to_csv(path + string.join(row) + '.csv')
        else:
          print("Stock not the full time on exchange", str(row))
        t += 1

    #Download Benchmark
    
    row = "QQQ"
    string = ''
    path = '/home/jj/Stocks/Data/QQQ/'
    hist = yfin.download(string.join(row), from1, to)
    hist.to_csv(path + string.join(row) + '.csv')
    

[*********************100%***********************]  1 of 1 completed

ADBE
AMD



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ABNB
ALNY


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


XEL


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ZS


[*********************100%***********************]  1 of 1 completed


In [15]:
if dataset == "QQQ":
    DIR = "/home/jj/Stocks/Data/QQQ/"

    def load_data(directory):
        files = os.listdir(directory)
        #print(files)
        stock_series = []
        cur = None
        cur_keys = []

        def get_data(file):
            path = f"{directory}/{file}"    
            df = pd.read_csv(path)

            df = df.drop([0,1])
            df = df.rename(columns={"Price":"Date"})
            df = df.set_index("Date")
            df = df[["Close"]]

            return df


        for file in files:
            new_one = get_data(file)
            cur_keys.append(file.replace(".csv", ""))
            stock_series.append(new_one)
            
        
        data = pd.concat(stock_series, axis = 1, keys = cur_keys)     
        data.vbt.drop_levels(-1, inplace=True)    
        return data

    data = load_data(DIR)
    qqq_rank = data["QQQ"]
    data = data.drop(columns=["QQQ"])
    data.dropna(subset = data.columns, how = 'all', inplace = True)
elif dataset == "SNP":
    DIR = "/home/jj/Stocks/Data/snp500_cleaned/"
    def load_data(directory):
        files = os.listdir(directory)
        stock_series = []
        cur = None
        cur_keys = []
        def get_data(file):
            path = f"{directory}/{file}"    
            df = pd.read_csv(path)
            try:
                df = df.set_index("Date")
            except Exception as e:
                print(file)
            return df


        for file in files:
            new_one = get_data(file)
            cur_keys.append(file.replace(".csv", ""))
            stock_series.append(new_one)
        
        
        data = pd.concat(stock_series, axis = 1, keys = cur_keys)
        data.vbt.drop_levels(-1, inplace=True)
        
        return data

    data = load_data(DIR)
    ##Fill nans with either previous value or next value
    for col in data.columns:
        data[col] = data[col].ffill()
        data[col] = data[col].bfill()
    
    data.index = pd.to_datetime(data.index)
    
data = data.astype(float)
data = data[data.index > "2010-06-10"] #remove everything else
#data = data[data.index < "2023-09-02"]

In [16]:

data.shape

(481, 101)

In [17]:
def momentum(data, lookback_period, roc_period):
    
    shifted_values = np.zeros(data.shape)
    shifted_values[:shifted_values.shape[0] - roc_period, :] = data[roc_period:, :]
    
    roc = ((shifted_values - data) / data) * 100
    roc = roc[:roc.shape[0] - roc_period, :]
    roc = np.ceil(roc)
    const = np.full(roc.shape, 0)
    roc = np.maximum(const, roc)
    const = np.full(roc.shape, 1)
    roc = np.minimum(const, roc)
    
    
    xxx = np.vstack([np.arange(lookback_period), np.ones(lookback_period)]).T
    values = np.zeros((data.shape[0] - lookback_period, data.shape[1]))  
    correl = np.zeros((data.shape[0] - lookback_period, data.shape[1]))  
    sortino = np.zeros((data.shape[0] - lookback_period, data.shape[1]))

    def sortino_ratio(series, N):
        mean = np.mean(series, axis = 1) * N
        for i in range(series.shape[0]):
            row = series[i]
            std_neg = np.std(row) * np.sqrt(N)
            mean[i] = mean[i] / std_neg
        
        return mean 
    
    for i in range(data.shape[1]):
        cur_data = np.log(data[:, i])

        # Sortino Ratio
        # pct_change = np.zeros(data[:, i].shape)
        # pct_change[1:] = np.diff(data[:, i]) / data[:, i][:-1] * 100
        # print(data[:, i].shape, pct_change.shape)
        
        view = np.lib.stride_tricks.sliding_window_view(cur_data, (lookback_period,))[1:, :]
        lin_reg=(np.linalg.inv(xxx.T @ xxx) @ (xxx.T) @ view.T)
                
        # slope
        roll_mat = lin_reg[0]
        
        #intercept
        lin_reg_b = lin_reg[1]

        x = view
        
        line = [i for i in range(len(view[0]))]
        m = np.tile(roll_mat[:, np.newaxis], x.shape[1])
        n = np.tile(lin_reg_b[:, np.newaxis], x.shape[1])
        y = np.multiply(line,m) + n

        #correlation
        samples = x.shape[1]
        centered_x = x - np.mean(x, axis=1, keepdims=True)
        centered_y = y - np.mean(y, axis=1, keepdims=True)
        cov_xy = np.mean(centered_x * centered_y, axis=1)
        var_x = np.mean(centered_x**2, axis=1)
        var_y = np.mean(centered_y**2, axis=1)
        corrcoef_xy = cov_xy / (np.sqrt(var_x *var_y))
        
        correl[:,i] = corrcoef_xy
        values[:, i] = roll_mat
        
        # Sortino part 2
        # pct_change_view = np.lib.stride_tricks.sliding_window_view(pct_change, (lookback_period,))[1:, :]
        # temp = sortino_ratio(pct_change_view, lookback_period)
        # sortino[:, i] = temp
        
    values = 1 + values
    values = np.power(values, 252)

    if roc.shape[0] - values.shape[0] >= 0:
        new_shape = roc.shape[0] - values.shape[0]
        roc = roc[new_shape:, :]
    else:
        new_shape = values.shape[0] - roc.shape[0]
        values = values[new_shape:, :]
        correl = correl[new_shape:, :]
        sortino = sortino[new_shape:, :]

    # Final calculation  
    res = roc * values #* correl
    
    original = np.full((data.shape[0], data.shape[1]), np.nan)
    original[original.shape[0] - res.shape[0]:, :] = res
    return original


def bi_weekly(ranks, data):
    data.index = pd.to_datetime(data.index)
    monday_data = data[data.index.weekday == 0]
    bi_weekly_data = monday_data.iloc[::2, :]
    
    
    ranks.index = pd.to_datetime(ranks.index)
    monday_ranks = ranks[ranks.index.weekday == 0] ## 0 is Monday
    bi_weekly_ranks = monday_ranks.iloc[::2, :]
    return bi_weekly_ranks, bi_weekly_data
    

def strategy(ranks, stock_selected):
    ##For multi level index columns
    size = np.full(ranks.shape, fill_value=np.nan)
    np_ranks = ranks
    n_stocks = bi_weekly_data.shape[1]
    for i in range(0, np_ranks.shape[1] // n_stocks):
        cur_batch = np_ranks[:, i * n_stocks:(i + 1) * n_stocks]
        
        argsort_indices = np.argsort(cur_batch, axis = 1) ##sort across row to get the best 2 stocks
        max_indices = argsort_indices[:, -1 * stock_selected :]
                
        ##skip those where ranks are not known , so first lookback period entries
        last_row_with_nan = np.where(np.isnan(cur_batch))[0][-1]
        ##Generate size matrix, where share is 0.95 / number of stocks, among max amount number of stocks
        for row, entry in enumerate(max_indices):
            if row <= last_row_with_nan: continue
            prev = row - 1
            if prev == last_row_with_nan + 1:
                ##initial buy
                entry = [(i * n_stocks) + x for x in entry]
                size[row, entry] = 0.95 / stock_selected ##OPEN current ones
            elif prev > last_row_with_nan + 1:
                entry = [(i * n_stocks) + x for x in entry]
                
                prev_entries = max_indices[prev]
                prev_entries = [(i * n_stocks) + x for x in prev_entries]
                if prev_entries == entry:
                    continue

                size[row, prev_entries] = 0 ##CLOSE the previous ones

                size[row, entry] = 0.95 / stock_selected ##OPEN current ones
    
    return size

In [18]:
logging = True if simulation_type == "single" else False
@njit
def pre_segment_func_multi_nb(c, price, sizes_np):
    """Perform rebalancing at a timestamp."""

    for col in range(c.from_col, c.to_col):
        # Target percentage requires knowing the value of all assets in the group
        # Let's calculate this value using the current order price (remove this to use previous close)
        c.last_val_price[col] = nb.get_col_elem_nb(c, col, price)
        
    # Fill the size array
    size = sizes_np[c.i]
    # Sort by order value to first sell the current positions to release funds
    order_value_out = np.empty(c.group_len, dtype=np.float_)
    nb.sort_call_seq_nb(c, size, 5, 0, order_value_out)
    
    # Forward size to order_func_nb
    return (size,)

@njit
def order_func_multi_nb(c, size, price):
#     print("here/")
    """Create an order. Gets only executed if the current segment is active."""
    return nb.order_nb(
        size=nb.get_elem_nb(c, size),
        price=nb.get_elem_nb(c, price),
        size_type = 5,
        direction = 0,
        fixed_fees = 7,
        log=logging,
    )

In [19]:
def change_data_like_weights(data, weights):
    temp = data.copy()
    repeats_n = weights.shape[1] // data.shape[1]
    mutli_index = weights.columns
    temp = temp.to_numpy()
    temp = np.tile(temp, repeats_n)
    assert temp.shape == weights.shape
    
    return pd.DataFrame(data = temp, index = data.index, columns = mutli_index)

In [20]:
indicator = vbt.IndicatorFactory(class_name = "momentum_indicator",
                            short_name = "momentum",
                            input_names=["data"],
                            param_names=["lookback_period","roc_period"],
                            output_names=["momentum"]
                             ).from_apply_func(
                                momentum,
                                lookback_period = single_params["lookback_period"],
                                roc_period = single_params["roc_period"]
                             )

strategy = vbt.IndicatorFactory(class_name = "momentum_indicator",
                                short_name = "strategy",
                                input_names=["ranks"],
                                param_names=["stocks"],
                                output_names=["weights"]
                                 ).from_apply_func(
                                    strategy,
                                    stocks = single_params["stocks"]
                                    
                                 )

In [21]:
res = indicator.run(data)

In [22]:
def generate_args():
    lookback_periods = []
    orcs = []
    
    for x in range(multi_params["min_lookback_period"], multi_params["max_lookback_period"] +1, multi_params["step"]):
        for y in range(multi_params["min_roc_period"], multi_params["max_roc_period"] + 1, multi_params["step"]):
            lookback_periods.append(x)
            orcs.append(y)
        
    return lookback_periods, orcs

In [23]:
if simulation_type == "single":
    ##Get ranks
    results = indicator.run(data)

    ##Get biwebi_weekly ranks
    bi_weekly_ranks, bi_weekly_data = bi_weekly(results.momentum, data)
    #del results
    ##Generate sizes
    strategy_res = strategy.run(bi_weekly_ranks)
    # Define rebalancing dates (= active segments)
    segment_mask = pd.Series(True, index=strategy_res.weights.index)
    price = bi_weekly_data  #actual prices
    group = True
    
else:
    lookback_periods, rocs = generate_args()
    results = indicator.run(data, lookback_periods, rocs)
    bi_weekly_ranks, bi_weekly_data = bi_weekly(results.momentum, data)
    del results
    strategy_res = strategy.run(bi_weekly_ranks, [i for i in range(multi_params["min_stocks"],
                                                                                multi_params["max_stocs"] +1)])
    price = change_data_like_weights(bi_weekly_data, strategy_res.weights)
    group = [0,1,2]

In [24]:
# Define rebalancing dates (= active segments)
segment_mask = pd.Series(True, index=strategy_res.weights.index)  
# Define and run the simulation

pf = vbt.Portfolio.from_order_func(
    price,
    order_func_multi_nb,
    vbt.Rep('price'),  # *args passed to order_func_nb
    pre_segment_func_nb=pre_segment_func_multi_nb,
    pre_segment_args=(
        vbt.Rep('price'),
        strategy_res.weights.to_numpy(),
        ),
    broadcast_named_args=dict(  # broadcast to a single shape
        price=price,
    ),
    group_by=group,
    segment_mask=segment_mask,
    init_cash = 10000000,
    freq="1W",
    cash_sharing=True # share cash within each split
)

In [25]:
if simulation_type == "single":
    print(pf.stats())
elif simulation_type == "multi":
    fig = pf.total_profit().vbt.heatmap(
        x_level='momentum_roc_period', y_level='momentum_lookback_period', slider_level="strategy_stocks", symmetric=True,
        trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='0.0')))
    fig.show()

Start                         2023-12-04 00:00:00
End                           2025-11-03 00:00:00
Period                          322 days 00:00:00
Start Value                            10000000.0
End Value                         21244819.919371
Total Return [%]                       112.448199
Benchmark Return [%]                    88.174907
Max Gross Exposure [%]                  96.815613
Total Fees Paid                              70.0
Max Drawdown [%]                        27.795807
Max Drawdown Duration            28 days 00:00:00
Total Trades                                    6
Total Closed Trades                             4
Total Open Trades                               2
Open Trade PnL                     7317902.133367
Win Rate [%]                                 75.0
Best Trade [%]                          67.087799
Worst Trade [%]                        -15.033343
Avg Winning Trade [%]                   46.344445
Avg Losing Trade [%]                   -15.033343


In [26]:
if simulation_type == "multi":
    pf.total_profit(group_by=False)
    plt.offline.plot(fig, filename='heatmap.html')
    pf[(1, 100, 100)].plot() ##cumulative returns

In [27]:
if simulation_type == "single":
    pf.orders.records_readable.to_csv("orders.csv")
    pf.logs.records.to_csv("logs.csv")

In [28]:
pf.stats(agg_func=None).to_excel("stats.xlsx")

In [29]:
if simulation_type == "single":
    pf.plot_drawdowns()
else:
    pf[(1, 200, 250)].plot_drawdowns()

In [30]:
pf.qs.html_report(output='/home/jj/Stocks/ts.html')
#pf.qs.full_report()

ValueError: Invalid frequency: ME

In [ ]:
if simulation_type == "single":
    holdings1 = pf.assets()[-1:]
    for col in holdings1.columns:
        if holdings1[col].max() > 0:
            column_header = col
            break
    print(holdings1[column_header])

In [ ]:
pd.set_option('display.max_rows', 5000)
pd.options.display.max_seq_items = 5000

In [ ]:
results.momentum.tail().sort_values(by = results.momentum.tail().index[-1], axis = 1, ascending = False)

In [ ]:
(data[["PLTR", "APP", "MSTR", "DASH"]].pct_change()+1).cumprod().plot()

In [ ]:
type(results.momentum)

In [ ]:
top_five = results.momentum.iloc[-1].nlargest(5)

In [ ]:
columns= top_five.index

In [ ]:
results.momentum[columns].plot()

In [ ]:
slope = pd.DataFrame(np.gradient(results.momentum, axis= 0), results.momentum.index, results.momentum.columns)

In [ ]:
slope[columns].plot()

In [ ]:
results.momentum.plot()

In [ ]:
#pf.assets()